In [ ]:
import torch
import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import os
from ruamel.yaml.scalarfloat import ScalarFloat
from config.synthetic_config.config_tauMLP_synthetic import get_config
from torch.utils.data import DataLoader
from lib.datasets import mnist, maze, protein, synthetic
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.dataset_utils as dataset_utils
from lib.datasets.metrics import eval_mmd

def main():
    
    script_dir = os.path.dirname(os.path.realpath(__file__))
    save_location = os.path.join(script_dir, "SavedModels/Synthetic/")
    dataset_location = os.path.join(script_dir, cfg.data.location)

    # creating paths
    path = 'SavedModels/Synthetic/' # 'SavedModels/MAZE/' 'SavedModels/MNIST/'
    date = '2023-11-16' # 2023-10-30 'Hollow-2023-10-29'
    config_name = 'config_001_tauMLP.yaml' # 'config_001_maze.yaml' 'config_001_rate001.yaml'
    model_name = 'model_199999_tauMLP.pt' # 'model_55999_rate001.pt' 'model_5999_maze.pt'

    #config_name = 'config_001_r07.yaml' 
    #model_name = 'model_84999_hollowr07.pt' 
    config_path = os.path.join(path, date, config_name)
    checkpoint_path = os.path.join(path, date, model_name)

    # creating models
    cfg = bookkeeping.load_config(config_path)
    cfg.sampler.name = 'ElboLBJF' #'ExactSampling' # ElboLBJF CRMTauL CRMLBJF
    cfg.logit_type = 'direct'
    cfg.sampler.num_corrector_steps = 10
    cfg.sampler.corrector_entry_time = ScalarFloat(0.1)
    cfg.sampler.num_steps = 750
    cfg.sampler.is_ordinal = True

    #print(cfg)
    device = torch.device(cfg.device)

    model = model_utils.create_model(cfg, device)
    print("number of parameters: ", sum([p.numel() for p in model.parameters()]))

    #modified_model_state = utils.remove_module_from_keys(loaded_state['model'])
    #model.load_state_dict(modified_model_state)
    #optimizer = optimizers_utils.get_optimizer(model.parameters(), cfg)
    optimizer = torch.optim.Adam(model.parameters(), cfg.optimizer.lr)

    sampler = sampling_utils.get_sampler(cfg)

    state = {"model": model, "optimizer": optimizer, "n_iter": 0}
    state = bookkeeping.load_state(state, checkpoint_path)
    state['model'].eval()

    dataset = dataset_utils.get_dataset(cfg, device, dataset_location)
    dataloader = DataLoader(dataset, batch_size=cfg.data.batch_size, shuffle=cfg.data.shuffle)

    n_samples = 1024
    n_rounds = 1
    mmd = eval_mmd(cfg, sampler, dataloader, n_rounds, num_samples=n_samples)
    print("MMD", mmd)
if __name__ == "__main__":
    main()
